In [11]:
import pandas as pd
import numpy as no
import datetime
from sqlalchemy import create_engine, inspect, text


In [12]:
df = pd.read_csv("ufo-sightings-transformed.csv")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80328 entries, 0 to 80327
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   80328 non-null  int64  
 1   Date_time                    80328 non-null  object 
 2   date_documented              80328 non-null  object 
 3   Year                         80328 non-null  int64  
 4   Month                        80328 non-null  int64  
 5   Hour                         80328 non-null  int64  
 6   Season                       80328 non-null  object 
 7   Country_Code                 80069 non-null  object 
 8   Country                      80069 non-null  object 
 9   Region                       79762 non-null  object 
 10  Locale                       79871 non-null  object 
 11  latitude                     80328 non-null  float64
 12  longitude                    80328 non-null  float64
 13  UFO_shape       

In [14]:
##Need to update the Date_time column datatype 
## Could drop the description column 
## Only one column for encounter duration is really needed, would seconds or minutes be better?
## Do we need date the encounter was documented? Or just date that encounter occured? 

In [15]:
df['Date_time'] = pd.to_datetime(df['Date_time'])

In [16]:
ufo_df = df.rename(columns = {'latitude' : 'Latitude', 'longitude' : 'Longitude'})

In [17]:
##Making the duration of encounter (seconds) column a minutes column instead 

ufo_df['Estimated_Encounter_Duration_Minutes'] = (ufo_df['length_of_encounter_seconds'] / 60).round(2)

In [18]:
ufo_df = ufo_df.drop(['date_documented', 'Country_Code', 'length_of_encounter_seconds', 'Encounter_Duration', 'Description'], axis=1)

In [19]:
ufo_df = ufo_df.rename(columns={'Unnamed: 0' : 'ID'})

ufo_df = ufo_df.set_index('ID')


ufo_df.head()

,Date_time,Year,Month,Hour,Season,Country,Region,Locale,Latitude,Longitude,UFO_shape,Estimated_Encounter_Duration_Minutes
ID,,,,,,,,,,,,
0,1949-10-10 20:30:00,1949,10,20,Autumn,United States,Texas,San Marcos,29.883056,-97.941111,Cylinder,45.00
1,1949-10-10 21:00:00,1949,10,21,Autumn,United States,Texas,Bexar County,29.384210,-98.581082,Light,120.00
2,1955-10-10 17:00:00,1955,10,17,Autumn,United Kingdom,England,Chester,53.200000,-2.916667,Circle,0.33
3,1956-10-10 21:00:00,1956,10,21,Autumn,United States,Texas,Edna,28.978333,-96.645833,Circle,0.33
4,1960-10-10 20:00:00,1960,10,20,Autumn,United States,Hawaii,Kaneohe,21.418056,-157.803611,Light,15.00


In [20]:
ufo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80328 entries, 0 to 80327
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Date_time                             80328 non-null  datetime64[ns]
 1   Year                                  80328 non-null  int64         
 2   Month                                 80328 non-null  int64         
 3   Hour                                  80328 non-null  int64         
 4   Season                                80328 non-null  object        
 5   Country                               80069 non-null  object        
 6   Region                                79762 non-null  object        
 7   Locale                                79871 non-null  object        
 8   Latitude                              80328 non-null  float64       
 9   Longitude                             80328 non-null  float64       
 10  UFO

In [21]:
ufo_df = ufo_df.dropna(subset=['Region', 'Locale', 'UFO_shape'])

In [22]:
ufo_df.head()

,Date_time,Year,Month,Hour,Season,Country,Region,Locale,Latitude,Longitude,UFO_shape,Estimated_Encounter_Duration_Minutes
ID,,,,,,,,,,,,
0,1949-10-10 20:30:00,1949,10,20,Autumn,United States,Texas,San Marcos,29.883056,-97.941111,Cylinder,45.00
1,1949-10-10 21:00:00,1949,10,21,Autumn,United States,Texas,Bexar County,29.384210,-98.581082,Light,120.00
2,1955-10-10 17:00:00,1955,10,17,Autumn,United Kingdom,England,Chester,53.200000,-2.916667,Circle,0.33
3,1956-10-10 21:00:00,1956,10,21,Autumn,United States,Texas,Edna,28.978333,-96.645833,Circle,0.33
4,1960-10-10 20:00:00,1960,10,20,Autumn,United States,Hawaii,Kaneohe,21.418056,-157.803611,Light,15.00


In [23]:
ufo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77676 entries, 0 to 80327
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Date_time                             77676 non-null  datetime64[ns]
 1   Year                                  77676 non-null  int64         
 2   Month                                 77676 non-null  int64         
 3   Hour                                  77676 non-null  int64         
 4   Season                                77676 non-null  object        
 5   Country                               77676 non-null  object        
 6   Region                                77676 non-null  object        
 7   Locale                                77676 non-null  object        
 8   Latitude                              77676 non-null  float64       
 9   Longitude                             77676 non-null  float64       
 10  UFO

In [24]:
engine = create_engine("sqlite:///UFO_data.sqlite")

conn = engine.connect()


In [25]:
ufo_df.to_sql(name = "UFO_data", con=conn, index = False, if_exists = "append")

77676

In [26]:
inspector = inspect(engine)

In [27]:
tables = inspector.get_table_names()

for table in tables:
	print(table)
	print("--------")
	columns = inspector.get_columns(table)
	for column in columns: 
		print(column['name'], column['type'])

	print()

UFO_data
--------
Date_time DATETIME
Year BIGINT
Month BIGINT
Hour BIGINT
Season TEXT
Country TEXT
Region TEXT
Locale TEXT
Latitude FLOAT
Longitude FLOAT
UFO_shape TEXT
Estimated_Encounter_Duration_Minutes FLOAT



In [28]:
engine.dispose()